In [1]:
#%set_env SDSS_LOCAL_SAS_MIRROR=/hd0/data/APOGEE
%set_env RESULTS_VERS=l31c.2
#%set_env APOGEE_APOKASC_REDUX=v6.2a
#%env

env: RESULTS_VERS=l31c.2


In [2]:
#Change DR
#from apogee.tools.path import change_dr
#change_dr('14')

In [3]:
#from apogee.tools import download
#download.ferreModelLibrary(lib='GK',pca=True,sixd=True,unf=False,convertToBin=True)

In [4]:
import apogee
%matplotlib inline
import numpy as np
import apogee.tools.read as apread
import apogee.spec.plot as splot

RuntimeError: SDSS_LOCAL_SAS_MIRROR environment variable needs to be set to use the 'apogee' module

In [ ]:
import apogee.tools.read as apread
allStar= apread.allStar(rmcommissioning=True,main=False,ak=True, akvers='targ',adddist=False)

In [ ]:
from apogee.modelspec import ferre


mspec= ferre.interpolate(4750.,2.5,-0.1,0.1,0.,0.)
splot.waveregions(mspec)

For TYC 3559 the useful info is:
- https://dr12.sdss.org/infrared/spectrum/search?id=19164
- APOGEE_ID = 2M20041135+4505154
- Location_IC = 4462

In [ ]:
allStar= apread.allStar(rmcommissioning=True,rmdups=True)

In [ ]:
#param = ferre.fit(4462,'2M20041135+4505154',lib='GK',pca=True,sixd=True)
#print(param)

In [ ]:
#Get the actual value
fparam = allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['FPARAM']
print(fparam)

In [ ]:
#Now include the Cannon
cparam = ferre.fit(4462,'2M20041135+4505154',lib='GK',pca=True,sixd=True,initcannon=True)
print(param)

Let's try converting to ASPCAP values using http://adsabs.harvard.edu/cgi-bin/nph-data_query?bibcode=2015AJ....150..148H&db_key=AST&link_type=ABSTRACT&high=5a748cc7a900541

In [ ]:
def dr12teff(raw_teff):
    '''This function applies the DR12 calibration to Effective Temperature'''
    #Add in limits for applicable teffs
    if raw_teff < 3000:
        raw_teff = 3000
    if raw_teff > 6000:
        raw_teff = 6000
    calib_teff = raw_teff - 0.0034*(raw_teff-4500)+87.9
    return calib_teff

In [ ]:
teff_aspcap = float(allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['TEFF'])
print("Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(fparam[0,0],param[0,0],cparam[0,0]))
print("ASPCAP: {:.3f}, Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(teff_aspcap,dr12teff(fparam[0,0]),
                                                            dr12teff(param[0,0]),dr12teff(cparam[0,0])))

Now do it on log g.

In [ ]:
def dr12logg(raw_logg):
    '''This function applies the DR12 calibration to RGB Log g (do not use on RC stars).'''
    calib_logg = raw_logg + 0.14*raw_logg-0.588
    return calib_logg

In [ ]:
logg_aspcap = float(allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['LOGG'])
print("Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(fparam[0,1],param[0,1],cparam[0,1]))
print("ASPCAP: {:.3f}, Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(logg_aspcap,dr12logg(fparam[0,1]),
                                                            dr12logg(param[0,1]),dr12logg(cparam[0,1])))

In [ ]:
def dr12M_H(raw_M_H,teff):
    '''This function applies the DR12 calibration [M/H] log g <= 3.8'''
    
    #I don't know whether Teff is calibrated or not
    if teff < 3800:
        teff = 3800
    if teff > 5250:
        teff = 5250
    

    calib_M_H = raw_M_H +0.056*(teff -4500)/1000
    #Now add outside calibration
    extern_M_H = calib_M_H + 0.0260 + 0.255*calib_M_H + 0.062*calib_M_H**2
    
    return extern_M_H

def dr12alpha(raw_alpha,teff):
    '''This function applies the DR12 calibration [M/H] log g <= 3.8'''
    
    #I don't know whether Teff is calibrated or not
    if teff < 3800:
        teff = 3800
    if teff > 5250:
        teff = 5250   

    calib_alpha = raw_alpha +0.028*(teff -4500)/1000
    
    return calib_alpha

In [ ]:
print(allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['PARAM'][0,6],dr12alpha(fparam[0,6],fparam[0,0]))
M_H_aspcap = float(allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['METALS'])
print("Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(fparam[0,3],param[0,3],cparam[0,3]))
print("ASPCAP: {:.3f}, Fparam: {:.3f}, Fit: {:.3f}, Cannon: {:.3f}".format(M_H_aspcap,dr12M_H(fparam[0,3],fparam[0,0]),
                                                            dr12M_H(param[0,3],fparam[0,0])
                                                            ,dr12M_H(cparam[0,3],fparam[0,0])))

In [ ]:
alpha_aspcap = float(allStar[(allStar['APOGEE_ID'] == '2M20041135+4505154')]['ALPHAFE'])
print("Fparam: {:.4f}, Fit: {:.4f}, Cannon: {:.4f}".format(fparam[0,6],param[0,6],cparam[0,6]))
print("ASPCAP: {:.4f}, Fparam: {:.4f}, Fit: {:.4f}, Cannon: {:.4f}".format(alpha_aspcap,dr12alpha(fparam[0,6],fparam[0,0]),
                                                            dr12alpha(param[0,6],fparam[0,0])
                                                            ,dr12alpha(cparam[0,6],fparam[0,0])))